In [2]:
import numpy as np    #导入numpy包

In [3]:
"""
函数说明:加载数据

Parameters:
    fileName - 文件名
Returns:
    dataMat - 数据列表
    labelMat - 标签列表
"""
def loadDataSet(fileName):
    dataMat = []; labelMat = []                #创建数据列表，标签列表
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]), float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat,labelMat

In [1]:

"""
函数说明:SMO辅助函数

Parameters:
    i - 第一个alpha的下标
    m - 所有alpha的数目
Returns:
    j - 不等于i的下标
"""
def selectJrand(i,m):
    j = i
    while(j == i):
        j = int(np.random.uniform(0,m))
    return j        #选择一个不等于i的j返回

"""
函数说明:SMO辅助函数
调整大于H或小于L的alpha的值
Parameters:
    aj - alpha的值
    H - 限定alpha的最大值
    L - 限定alpha的最小值
Returns:
    aj - 范围限定于[L,H]间的alpha的值
"""
def clipAlpha(aj, H, L):
    if aj > H:
        aj = H
    if L > aj:
        aj = L
    return aj

In [4]:
"""
函数说明:简化版SMO函数

Parameters:
    dataMatIn - 数据列表
    classLabels - 标签列表
    C - 惩罚参数
    toler - 容错率
    maxIter - 最大迭代次数
Returns:
    b -
    alphas -  
"""
def somSimple(dataMatIn, classLabels, C, toler, maxIter):
    dataMatrix = np.mat(dataMatIn); labelMat = np.mat(classLabels).transpose()     #将训练数据集和标签转换为numpy矩阵
    b = 0; m,n = np.shape(dataMatrix)       #初始化b为0，返回数据集的行和列
    alphas = np.mat(np.zeros((m,1)))        #初始化alphas
    iter = 0                                #初始化迭代次数
    while(iter < maxIter):
        alphaPairsChanged = 0
        for i in range(m):
            #步骤1：计算误差  Ei = f(xi)-yi
            fXi = float(np.multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T))+b
            Ei = fXi - float(labelMat[i])
            # 优化alpha，更设定一定的容错率
            if((labelMat[i]*Ei<-toler)and(alphas[i]<C))or((labelMat[i]*Ei>toler)and(alphas[i]>0)):
                # 随机选择另一个与alpha_i成对优化的alpha_j
                j = selectJrand(i,m)
                # 步骤1：计算误差Ej Ej = f(xj)-yj
                fXj = float(np.multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[j,:].T))+b
                Ej = fXj - float(labelMat[j])
                # 保存更新前的aplpha值，使用深拷贝
                alphaIold = alphas[i].copy()
                alphaJold = alphas[j].copy()
                #步骤2：计算L和H
                if(labelMat[i] != labelMat[j]):
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C+alphas[j]-alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i]-C)
                    H = min(C, alphas[j] + alphas[i])
                if L == H: print("L==H"); continue
                #步骤3：计算eta
                eta = 2.0 * dataMatrix[i,:]*dataMatrix[j,:].T - dataMatrix[i,:]*dataMatrix[i,:].T \
                       - dataMatrix[j, :] * dataMatrix[j, :].T
                if eta >=0: print("eta>=0");continue
                #步骤4：更新alpha_j
                alphas[j] -= labelMat[j]*(Ei - Ej)/eta
                #步骤5：修剪alpha_j
                alphas[j] = clipAlpha(alphas[j], H, L)
                if(abs(alphas[j]-alphaJold)<0.00001):print("j not moving enough");continue
                #步骤6：更新alpha_i
                alphas[i] += labelMat[j]*labelMat[i]*(alphaJold-alphas[j])
                #步骤7：更新b1和b2
                b1 = b - Ei - labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T \
                 - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i,:]*dataMatrix[j,:].T
                b2 = b - Ej - labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[j,:].T \
                 - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j,:]*dataMatrix[j,:].T
                #根据b1和b2确定b
                if(0<alphas[i]) and (C > alphas[i]): b= b1
                elif(0<alphas[j]) and (C>alphas[j]):b = b2
                else: b = (b1+b2)/2.0
                #统计优化次数
                alphaPairsChanged += 1
                #打印统计信息
                print("iter: %d i:%d, pairs changed %d" % (iter, i, alphaPairsChanged))
        #更新迭代次数
        if(alphaPairsChanged == 0): iter += 1
        else: iter = 0
        print('iteration number: %d' % iter)
    return b,alphas

In [7]:
dataArr,labelArr = loadDataSet('data/testSet.txt')
b,alphas = somSimple(dataArr,labelArr, 0.6,0.001,40)
print(b)
print(alphas[alphas>0])

L==H
L==H
iter: 0 i:2, pairs changed 1
j not moving enough
iter: 0 i:5, pairs changed 2
iter: 0 i:8, pairs changed 3
L==H
L==H
j not moving enough
L==H
j not moving enough
iter: 0 i:76, pairs changed 4
j not moving enough
L==H
L==H
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iter: 0 i:10, pairs changed 1
j not moving enough
j not moving enough
L==H
L==H
iter: 0 i:35, pairs changed 2
j not moving enough
L==H
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
L==H
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
L==H
iteration num

iteration number: 6
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 7
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 8
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iter: 8 i:54, pairs changed 1
j not moving enough
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
j not movi

iteration number: 5
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 6
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 7
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 8
j not moving enough
j not moving enough
j not moving enough
iter: 8 i:52, pairs changed 1
iter: 8 i:54, pairs changed 2
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 3


j not moving enough
j not moving enough
j not moving enough
iteration number: 3
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 4
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 5
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 6
j not moving enough
j not moving enough
j not moving enough
iter: 6 i:55, pairs changed 1
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
iter: 1 i:29, pairs changed 1
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
j not moving enough
j not moving enough


iteration number: 0
L==H
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
iter: 1 i:29, pairs changed 1
j not moving enough
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
iter: 1 i:52, pairs changed 1
j not moving enough
iter: 1 i:55, pairs changed 2
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
j not moving enough
iteration number: 3
j not moving enough
j not moving enough
j not moving enough
iteration number: 4
j not moving enough
j not moving enough
j not moving enough
iteration number: 5
j not moving enough
j not moving enough
j not moving enough
iteration number: 6
j not moving enough
j not moving enough
j not moving enough
iteration number: 7
j not moving enough
j not

j not moving enough
j not moving enough
iteration number: 10
j not moving enough
j not moving enough
iter: 10 i:55, pairs changed 1
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
j not moving enough
iteration number: 3
j not moving enough
j not moving enough
j not moving enough
iteration number: 4
iter: 4 i:17, pairs changed 1
j not moving enough
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
j not moving enough
iteration number: 3
j not moving enough
j not moving enough
j not moving enough
iteration number: 4
j not moving enough
j not moving enough
j not moving enough
iteration number: 5
j not moving enoug